# Import data and preprocessing. replace the missing value with mean and deal with categorical data.

In [16]:
import csv
import numpy
import scipy
from copy import deepcopy
import pandas as pd

import random
import math
from operator import itemgetter
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D



data_list=[]
y=[]
data=csv.reader(open('/Users/wendy/Documents/2017 Fall/CS 534/HW4/allhyper.data'))
for row in data:
    data_list.append(row)
features=deepcopy(data_list[0])
data_list.pop(0)

####convert string to int
for i in range(len(data_list)):
    if data_list[i][29]=='negative.':
        data_list[i][29]=0
    else:
        data_list[i][29]=1
############
for i in range(len(data_list)):
    if data_list[i][1]=='F':
        data_list[i][1]=1
    elif data_list[i][1]=='M':
        data_list[i][1]=2
    else:
        data_list[i][1]=0
index_list=(2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,20,22,24,26)
for i in index_list:
    for j in range(len(data_list)):
        
        if data_list[j][i]=='f':
            data_list[j][i]=0
        else:
            data_list[j][i]=1
############
class_mean={}
def calculate_mean(n):
    sum=0
    index=0
    for i in range(len(data_list)):
        if data_list[i][n]!='NA':
            sum+=float(data_list[i][n])
            index+=1
    return sum/index
class_mean[0]=calculate_mean(0)
class_mean[17]=calculate_mean(17)
class_mean[19]=calculate_mean(19)
class_mean[21]=calculate_mean(21)
class_mean[23]=calculate_mean(23)
class_mean[25]=calculate_mean(25)
print class_mean
feature_list=(0,17,19,21,23,25)
for i in feature_list:
    for j in range(len(data_list)):
        if data_list[j][i]=='NA':
            data_list[j][i]=class_mean[i]
################            
whole_data=numpy.array(data_list)

trainx=whole_data[0:int(0.7*len(whole_data)),0:26].astype(float)
trainy=whole_data[0:int(0.7*len(whole_data)),29].astype(float)
testx=whole_data[int(0.7*len(whole_data)):len(whole_data),0:26].astype(float)
testy=whole_data[int(0.7*len(whole_data)):len(whole_data),29].astype(float)


{0: 51.8442300821722, 17: 4.672150238473764, 19: 2.0249661399548584, 21: 109.07240061162081, 23: 0.9979121054734302, 25: 110.78798403193613}


# Use cross validation to find the optimal parameter

In [97]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
r=numpy.logspace(-5,5,num=20)
h=numpy.arange(10)
best_score=0
for a in r:
    for b in h:
        clf = MLPClassifier(solver='lbfgs', alpha=a,hidden_layer_sizes=(b+1,), random_state=1)
        clf.fit(trainx,trainy)
        scores = cross_val_score(clf, trainx,trainy, scoring='accuracy',cv=5)
        score_mean=max(scores)
        if score_mean>best_score:
            best_score=score_mean
            best_alpha=a
            best_hidden=b+1
print best_alpha,best_hidden

0.000379269019073 10


# claculate the accuracy

In [90]:
clf = MLPClassifier(solver='lbfgs', alpha=0.000379269019073,hidden_layer_sizes=(10,), random_state=1)
clf.fit(trainx,trainy)
prediction=clf.predict(testx)
err=0
for i in range(len(prediction)):
    if prediction[i]!=testy[i]:
        err+=1
print  1-float(err)/len(testy)     

0.976218787158


# compare the accuracy ,f1,f2score between Neuron Network, linear SVM and Random forset

In [ ]:
# from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
f1=f1_score(testy, prediction, average='weighted')  
f2=fbeta_score(testy, prediction, average='weighted',beta=0.5)
print f1,f2


|         |accuracy|f1 score|f2 score|
| ------------- |:-------------:|
|NN|0.976218787158|0.972037778073|0.971954609589|
|linear SVM|0.9872383869321082|0.977142080437|0.986775456095|
|Random forest|0.985493230174|0.978256985439|0.977782966194|

As shown in the table, SVM and Random forset overall has a better performance than NN, higher accuracy,higher f1 score and f2 score.However HH has less parameter to train and requires less computational time.